In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
import numpy as np


In [3]:
m = np.random.random((6, 6))
M = 0.5*(m + m.T)
M_tf = tf.convert_to_tensor(M, name='M')
x = tf.placeholder(tf.float64, (1, 6), name='x')

In [4]:

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(x, 50, name='hidden1',
                              activation='elu')

    output = tf.layers.dense(hidden1, 6, name='outputs')

In [5]:


def rayleigh(M, x):
    return tf.divide(tf.matmul(tf.matmul(x,M),tf.transpose(x)), tf.matmul(x,tf.transpose(x)))
    
    
with tf.name_scope('cost'):
    cost = -rayleigh(M_tf, output)  # -rayleigh(M_tf, output) for largest eigenvalue
                                    # rayleigh(M_tf, output) for smallest

In [6]:
l_rate = 0.001
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(l_rate)
    training_op = optimizer.minimize(cost)

In [7]:
init = tf.global_variables_initializer()

Execution:

In [8]:
x_in = 3*np.random.random((1,6))

print (x_in)
N = 4000
with tf.Session() as sess:
    init.run()
    for i in range(N):
        sess.run(training_op, feed_dict={x: x_in})
        x_in = output.eval(feed_dict={x: x_in}).reshape((1, 6))
        
        if i%100 == 0:
            print (cost.eval(feed_dict={x: x_in}))
print (x_in)



[[1.82526038 0.97014391 1.06146749 1.97721794 0.29107211 1.29238452]]
[[1.75711423 2.15706293 1.97424603 2.31209979 1.69594897 1.73469012]]


In [9]:
def eigval(x, M):
    return (x.T@M@x)/(x.T@x)

nn_eigval = eigval(x_in.reshape((6, 1)), M)
from numpy.linalg import eig
eig(M)[0], nn_eigval


(array([ 2.83387061,  1.07396395,  0.29153155, -0.43558309, -0.2391612 ,
        -0.03423075]), array([[2.83387061]]))